# PyPSA Heat Pump
This spike belongs to a topic I created in the PyPSA Google Group: https://groups.google.com/g/pypsa/c/oTMq0RWg6oI.
It is about modelling a heat pump which is connected to 2 buffer storages. 
The heat pump can't heat both storages simultaneously and is also limited to run either 100% or 0% - nothing in between.

The solution is inspired by Tom Brown's hint in the mentioned Google Group as well as on the PyPSA implementation of unit commitment for generators (https://github.com/PyPSA/PyPSA/blob/master/pypsa/opf.py).

## Network Setup

In [32]:
import matplotlib.pyplot as plt
from typing import List, Tuple
from pyomo.environ import (Var, Constraint, Binary)
import pypsa.linopt
from pypsa import Network
import pandas as pd
import pyomo.kernel as pmo

n = pypsa.Network()
snapshots = [1, 2, 3, 4]
n.set_snapshots(snapshots)

hp_h_bus = 'heat_bus'
hp_ww_bus = 'ww_bus'
e_bus = 'e_bus'

n.add('Bus', hp_h_bus)
n.add('Bus', hp_ww_bus)
n.add('Bus', e_bus)

n.add("Generator", f"PV1",
       bus=e_bus,
       committable=False,
       marginal_cost=1,
       p_nom=2)

n.add("Link",
      f"hp_h",
      bus0=e_bus,
      bus1=hp_h_bus,
      p_nom=1,
      efficiency=4)  # Todo: use different avg cop for heating and warm water

n.add("Link",
      f"hp_ww",
      bus0=e_bus,
      bus1=hp_ww_bus,
      p_nom=1,
      efficiency=4)

n.add("Store", name=f"h_buffer", bus=hp_h_bus, e_nom=8, e_initial=0, marginal_cost=2)
n.add("Store", name=f"ww_buffer", bus=hp_ww_bus, e_nom=8, e_initial=0, marginal_cost=2)

n.add("Load", f"h_load", bus=hp_h_bus, p_set=[0, 0, 4, 4])
n.add("Load", f"ww_load", bus=hp_ww_bus, p_set=[0, 0, 4, 0])

## Build PYOMO constraints for heat pump functionality (and solve)

In [33]:
def build_hp_functionality(hps: List[Tuple[str]], min_up_time):
    all_hp_parts = [hp_part for hp in hps for hp_part in hp]
    
    def extra_functionality(network,snapshots):
        model = network.model
        snapshots = network.snapshots

        model.link_status = Var(all_hp_parts, snapshots, within=Binary)

        # exclusivity rule - enforces that only one of both hp parts can run in a timestep
        for i, parts in enumerate(hps):
            part1, part2 = parts
            model.add_component(f"hp_{i}_part_exclusivity", Constraint(snapshots, 
                                                   rule=lambda model, i: model.link_status[(part1, i)] + model.link_status[(part2, i)] <= 1
                                                  ))

        # power bounds rule - Enforces min/max values for p and couples p to the status. If status is zero, min/max is zero. If status is 1, min/max is p_nom
        all_hp_parts_per_timestep = [(l, s) for s in network.snapshots for l in all_hp_parts]
        n.model.link_hp_max_power_rule = Constraint(all_hp_parts_per_timestep,
                                            rule=lambda model, l, i: model.link_p[(l, i)] <= model.link_status[(l, i)] * network.links.p_nom[l])
        n.model.link_hp_min_power_rule = Constraint(all_hp_parts_per_timestep,
                                            rule=lambda model, l, i: model.link_p[(l, i)] >= model.link_status[(l, i)] * network.links.p_nom[l])

        # min runtime rule - Enforces a minimal runtime for each hp part
        for link_i, link in enumerate(all_hp_parts):
            blocks = range(0,len(snapshots)-1)

            def gen_hp_min_runtime_rule(model,i):
                period = min(min_up_time,len(snapshots)-i)
                lhs = sum(network.model.link_status[link,snapshots[j]] for j in range(i,i+period))
                last_step = 0 if i == 0 else network.model.link_status[link,snapshots[i-1]]
                rhs = period*network.model.link_status[link,snapshots[i]] - period * last_step
                return lhs >= rhs

            network.model.add_component("link_up_time_{}".format(link_i),Constraint(blocks,rule=gen_hp_min_runtime_rule))
    
    return extra_functionality

# hp_pairs = [[f"s_{i}_hp_ww", f"s_{i}_hp_h"] for i in range(1, 2)]
hp_pairs = [[f"hp_ww", f"hp_h"]]
n.lopf(solver_name='cbc', extra_functionality=build_hp_functionality(hp_pairs, 2))


INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using cbc
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -4.0
  Upper bound: -4.0
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 16
  Number of binary variables: 8
  Number of integer variables: 8
  Number of nonzeros: 2
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
   

(<SolverStatus.ok: 'ok'>, <TerminationCondition.optimal: 'optimal'>)

## Display Results

In [36]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.DataFrame({l: [n.model.link_status[l, i].value for i in n.snapshots] for l in [f"hp_h", f"hp_ww"]})
n.links_t.p1
n.stores_t.e

,hp_h,hp_ww
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


,hp_h,hp_ww
1,-0.0,-4.0
2,-0.0,-4.0
3,-4.0,-0.0
4,-4.0,-0.0


,h_buffer,ww_buffer
1,0.0,4.0
2,0.0,8.0
3,0.0,4.0
4,0.0,4.0
